In [97]:
import matplotlib.pyplot as plt
import pygame
import numpy as np
import math
import random

In [98]:
##### CONSTANTES

f = 0.3
rb = 5
rt = 20
dt = 0.0
m = 1
g = 9.81
nbre_mouvements = 15
nb_gen = 100
mutation_rate = 0.005

In [99]:
###### FONCTIONS UTILES

def produitvectoriel(vect1,vect2):
    return (vect1[0]*vect2[1] - vect1[1]*vect2[0])

def produitscalaire(vect1,vect2):
    return(vect1[0]*vect2[0] + vect1[1]*vect2[1])



def print_game(partie, vitesses, angles):
    pygame.init()
    win = pygame.display.set_mode((700,700))
    pygame.display.set_caption("Le mini-golf de l'enfer")
    
    x_trou = partie.trou.position[0]
    y_trou = partie.trou.position[1]
    
    win.fill((0,128,0))
    for i in obstacles:
        pygame.draw.line(win, (0,0,0), i.depart, i.arrivee,10)
        
    pygame.draw.circle(win, (250,250,250), (int(partie.balle.position[0]),int(partie.balle.position[1])), rb) 
    pygame.draw.circle(win, (0,0,0), (x_trou,y_trou), rt) 
    pygame.display.update()
    run = True
    
    while run :
        pygame.time.delay(10)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

        keys = pygame.key.get_pressed()

        win.fill((0,128,0))
        for i in obstacles:
            pygame.draw.line(win, (0,0,0), i.depart, i.arrivee,10)
        pygame.draw.circle(win, (0,0,0), (x_trou,y_trou), rt) 
        pygame.draw.circle(win, (250,250,250), (int(partie.balle.position[0]),int(partie.balle.position[1])), rb)
        pygame.display.update()


        if keys[pygame.K_SPACE] :
            for vit,ang in zip(vitesses,angles) :
                if partie.is_over:
                       break
                pygame.time.delay(30)
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        run = False
                for i in obstacles:
                    pygame.draw.line(win, (0,0,0), i.depart, i.arrivee,10)
                pygame.draw.circle(win, (0,0,0), (x_trou,y_trou), rt) 
                new_pos = partie.deplacement([vit,ang])
                pygame.draw.circle(win, (250,250,250), (int(new_pos[0]),int(new_pos[1])), rb)
                pygame.display.update()
                if partie.is_over:
                    vit = 0
                else :
                    vit-=0.5
                win.fill((0,128,0))



    print("fin")
    pygame.display.flip()
    pygame.quit()
    




In [100]:
##### CLASSE JOUEUR


class Joueur:
    
    
    
    def __init__(self, array_trou,array_balle,obstacles, nbre_mouvements):
        self.is_best = False
        self.fitness = 0
        self.partie = Partie(array_trou,array_balle,obstacles)
        self.liste_mouvements = []
        self.liste_emplacements = [array_balle]
        for i in range(nbre_mouvements):
            v = random.random()*10
            t = random.random()*2*np.pi
            self.liste_mouvements.append([v,t])
            
    def evaluate_fitness(self):
        distance = np.sqrt((self.partie.trou.position[0] - self.partie.balle.position[0])**2 + (self.partie.trou.position[1] - self.partie.balle.position[1])**2 )
        self.fitness = distance
        
    def faire_moves(self):
        for i in self.liste_mouvements:
            self.liste_emplacements.append(self.partie.deplacement(i))
    
    def mutate(self):
        for i in self.liste_mouvements:
            if random.random() < mutation_rate:
                i[0] = random.random()*10
            if random.random() < mutation_rate:
                i[1] = random.random()*2*np.pi
                

In [101]:
###### CLASSE POPULATION

class Population:
    def __init__(self, size, arraytrou, arrayballe, obstacle, nbre_mouvement):
        self.liste_joueur = []
        for i in range(size):
            self.liste_joueur.append(Joueur(arraytrou, arrayballe, obstacles, nbre_mouvements))

    def mutate_all(self,arraytrou, arrayballe, obstacles, nbre_mouvements):
        for i in range(len(self.liste_joueur)):
            self.liste_joueur[i].partie = Partie(arraytrou,arrayballe,obstacles)
            nouveauJoueur = Joueur(arraytrou, arrayballe, obstacles, nbre_mouvements)
            for j in range(nbre_mouvements):
                nouveauJoueur.liste_mouvements[j] = self.liste_joueur[i].liste_mouvements[j]
            nouveauJoueur.mutate()
            self.liste_joueur[i].faire_moves()
            self.liste_joueur[i].evaluate_fitness()
            nouveauJoueur.faire_moves()
            nouveauJoueur.evaluate_fitness()
            if nouveauJoueur.fitness > self.liste_joueur[i].fitness :
                self.liste_joueur[i] = nouveauJoueur
                
                
    def print_partie(self,arraytrou, arrayballe, obstacles, nbre_mouvements):
        pygame.init()
        win = pygame.display.set_mode((700,700))
        pygame.display.set_caption("Le mini-golf de l'enfer")

        win.fill((0,128,0))
        for i in obstacles:
            pygame.draw.line(win, (0,0,0), i.depart, i.arrivee,10)


        while run :
            pygame.time.delay(10)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    run = False

            keys = pygame.key.get_pressed()

            win.fill((0,128,0))
            for i in obstacles:
                pygame.draw.line(win, (0,0,0), i.depart, i.arrivee,10)
            pygame.draw.circle(win, (0,0,0), (x_trou,y_trou), rt) 
            for i in self.liste_joueur :
                pygame.draw.circle(win, (250,250,250), (int(self.liste_joueur[i].partie.balle.position[0]),int(self.liste_joueur[i].partie.balle.position[1])), rb)
                pygame.display.update()


            if keys[pygame.K_SPACE] :
                for i in range(len(self.liste_joueur[i].liste.liste_mouvements)): 
                    pygame.time.delay(30)
                    for event in pygame.event.get():
                        if event.type == pygame.QUIT:
                            run = False
                    pygame.draw.circle(win, (0,0,0), (x_trou,y_trou), rt)
                    for obs in obstacles:
                        pygame.draw.line(win, (0,0,0), obs.depart, obs.arrivee,10)
                    for j in range(len(self.liste_joueur)):
                        pygame.draw.circle(win, (250,250,250), (int(self.liste_joueur[j].liste_mouvements[i][0]),int(self.liste_joueur[j].liste_mouvements[i][1])), rb)
                        pygame.display.update()
                    win.fill((0,128,0))



        print("fin")
        pygame.display.flip()
        pygame.quit()

In [102]:
#### CLASSE SEGMENT

class Segment:
    def __init__(self,array1,array2):
        self.depart = array1
        self.arrivee = array2
        
    def cross(self , segment):
        ab = [self.arrivee[0] - self.depart[0] , self.arrivee[1] - self.depart[1]]
        apbp = [segment.arrivee[0] - segment.depart[0] , segment.arrivee[1] - segment.depart[1]]
        abp = [segment.arrivee[0] - self.depart[0], segment.arrivee[1] - self.depart[1]]
        aap = [segment.depart[0] - self.depart[0], segment.depart[1] - self.depart[1] ]
        apb = [self.arrivee[0] - segment.depart[0] , self.arrivee[1] - segment.depart[1]]
        apa = [self.depart[0] - segment.depart[0] , self.depart[1] - segment.depart[1]]
        
        c1 = produitvectoriel(ab,apbp) != 0
        c2 = (produitvectoriel(ab,abp) * produitvectoriel(ab,aap))<=0
        c3 = (produitvectoriel(apbp,apb) * produitvectoriel(apbp,apa))<=0
        return(c1 and c2 and c3)
Segment1 = Segment([0,0],[0,1])    
Segment2 = Segment([1,0],[1,1]) 
Segment3 = Segment([-0.5,0.5],[0.5,0.5])
Segment1.cross(Segment1)

False

In [103]:

##### CLASSE TROU

class Trou:
    def __init__(self, array):
        self.position = array
 


In [104]:
##### CLASSE BALLE

class Balle:
    def __init__(self, array):
        self.position = array 


In [105]:
##### CLASSE PARTIE

class Partie:
    def __init__(self, arrayballe,arraytrou, obstacles):
        self.balle = Balle(arrayballe)
        self.trou = Trou(arraytrou)
        self.obstacles = obstacles
        self.is_dead = False
        self.nb_coups = 0
        self.reached_goal = False 
        
    def is_won(self):
        distance = np.sqrt((self.trou.position[0] - self.balle.position[0])**2 + (self.trou.position[1] - self.balle.position[1])**2 )
       # print(distance)
        if distance < rt :
            self.reached_goal = True
    
    def is_possible_move(self,segment_dep):
        liste_obstacles_cross = []
        for i in self.obstacles: # Pour tous les obstacles
            if i.cross(segment_dep): # Si un segment_dep et i se croisent alors le move n'est pas possible
                    liste_obstacles_cross.append(i)
        if len(liste_obstacles_cross) > 0 :
            liste_distances = []
            liste_inter = []
            for i in liste_obstacles_cross:
                if (segment_dep.depart[0] - segment_dep.arrivee[0] == 0):
                    a_i = (i.depart[1] - i.arrivee[1]) / (i.depart[0] - i.arrivee[0])
                    b_i = i.depart[1] - a_i *i.depart[0]
                    x_inter = segment_dep.depart[0]
                    y_inter = a_i * x_inter + b_i
                    liste_inter.append([x_inter , y_inter])
                    
                    
                elif (i.depart[0] - i.arrivee[0] == 0):
                    a_segment_dep = (segment_dep.depart[1] - segment_dep.arrivee[1]) / (segment_dep.depart[0] - segment_dep.arrivee[0])
                    b_segment_dep = segment_dep.depart[1] - a_segment_dep * segment_dep.depart[0]
                    x_inter = i.depart[0]
                    y_inter = a_segment_dep * x_inter + b_segment_dep
                    liste_inter.append([x_inter , y_inter])
                
                else :
                    a_segment_dep = (segment_dep.depart[1] - segment_dep.arrivee[1]) / (segment_dep.depart[0] - segment_dep.arrivee[0])
                    b_segment_dep = segment_dep.depart[1] - a_segment_dep * segment_dep.depart[0]
                    a_i = (i.depart[1] - i.arrivee[1]) / (i.depart[0] - i.arrivee[0])
                    b_i = i.depart[1] - a_i *i.depart[0]
                    x_inter = (b_i - b_segment_dep) / (a_segment_dep - a_i)
                    y_inter = a_i * x_inter + b_i
                    liste_inter.append([x_inter , y_inter])
                
                liste_distances.append(np.sqrt( (x_inter - self.balle.position[0])**2 + (y_inter - self.balle.position[1])**2))
            self.balle.position[0] = liste_inter[liste_distances.index(min(liste_distances))][0]
            self.balle.position[1] = liste_inter[liste_distances.index(min(liste_distances))][1]
                
            return False
        return True
    
    
    def deplacement(self,vitesse_initiale): # La vitesse sera de la forme [ ||v|| , theta)]
        distance =0.5*m/f*vitesse_initiale[0]**2
       # print(vitesse_initiale[0])
        segment_dep = Segment([self.balle.position[0],self.balle.position[1]],[self.balle.position[0] + np.cos(vitesse_initiale[1]) * distance,self.balle.position[1] + np.sin(vitesse_initiale[1]) * distance] )
       # print(self.is_possible_move(segment_dep))
        if self.is_possible_move(segment_dep):
            self.balle.position[0] += np.cos(vitesse_initiale[1]) * distance
            self.balle.position[1] += np.sin(vitesse_initiale[1]) * distance  
        else :
            self.is_dead = True
            #print("La balle est sur le mur")
        return (self.balle.position[0],self.balle.position[1])






In [106]:
#DEFINITION DE LA PARTIE
obstacles = []
# On ajoute les murs
obstacles.append(Segment([0,0],[0,700]))
obstacles.append(Segment([0,0],[700,0]))
obstacles.append(Segment([700,0],[700,700]))
obstacles.append(Segment([0,700],[700,700]))
# On ajoute les obstacles
obstacles.append(Segment([300,300],[900,300]))


array_balle = [500,500]
array_trou = [100,100]
partie = Partie(array_balle,array_trou, obstacles)

population = Population(100, array_trou, array_balle, obstacles, nbre_mouvements)

for i in range(nb_gen):
    print("Calculating generation : "+str(i))
    population.mutate_all(array_trou, array_balle, obstacles, nbre_mouvements)
    population.print_partie(array_trou, array_balle, obstacles)
    
    
print("La simulation est terminée.")
    

   






Calculating generation : 0


AttributeError: 'Population' object has no attribute 'print_game'